In [2]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

%run 01-model.ipynb

In [200]:
tryptamine = Metabolite(id = 'TRYP', compartment='c', name='tryptamine', formula='C10H12N2')

In [201]:
hydroxytryptamine = Metabolite(id='_4HTRYP', compartment='c', name='_4hydroxytryptamine', formula='C10H12N2O') 

In [202]:
norbaeocystin = Metabolite(id='NBC' , compartment='c', name='norbaeocystin', formula='C10H13N2O4P') 

In [203]:
psilocybin = Metabolite(id='psi', compartment='c', name='psilocybin', formula='C12H17N2O4P')

In [204]:
S_adenosyl_L_methionine = Metabolite(id='SAM', compartment='c', name='S-adenosyl-L-Methionine', formula='C15H22N6O5S')

In [205]:
S_adenosyl_L_homocysteine = Metabolite(id='SAH', compartment='c', name='S-adenosyl-L-homocysteine', formula='C14H20N6O5S' )

In [206]:
reaction_1 = Reaction('PsiD')  

In [207]:
reaction_2 = Reaction('PsiH')

In [208]:
reaction_3 = Reaction('PsiK')

In [209]:
reaction_4 = Reaction('PsiM')

In [210]:
reaction_1.add_metabolites(({model.metabolites.TRP: -1,  
                              tryptamine: 1, 
                              model.metabolites.CO2: 1, 
                            }))

In [211]:
reaction_2.add_metabolites({tryptamine: -1,
                              model.metabolites.O2: -1,
                              model.metabolites.H: -2, 
                              hydroxytryptamine: 1, 
                              model.metabolites.H2O: 1
                             })

In [212]:
reaction_3.add_metabolites({hydroxytryptamine: -1,
                              model.metabolites.ATP: -1,
                              norbaeocystin: 1, 
                              model.metabolites.ADP: 1, # our newly created metabolites
                              model.metabolites.H: 1
                             })

In [213]:
reaction_4.add_metabolites(({norbaeocystin: -1,
                             S_adenosyl_L_methionine: -2, 
                             psilocybin: 1, 
                             S_adenosyl_L_homocysteine: 2,
                             model.metabolites.H: 2
                             }))

In [214]:
model.add_reactions([reaction_1])
model.add_reactions([reaction_2])
model.add_reactions([reaction_3])
model.add_reactions([reaction_4])

In [215]:
print(reaction_1.build_reaction_string())
print(reaction_2.build_reaction_string())
print(reaction_3.build_reaction_string())
print(reaction_4.build_reaction_string())

TRP --> CO2 + TRYP
2 H + O2 + TRYP --> H2O + _4HTRYP
ATP + _4HTRYP --> ADP + H + NBC
NBC + 2 SAM --> 2 H + 2 SAH + psi


In [216]:
model.add_boundary(model.metabolites.TRYP, type='demand')
model.add_boundary(model.metabolites._4HTRYP, type='demand')
model.add_boundary(model.metabolites.NBC, type='demand')
model.add_boundary(model.metabolites.psi, type='demand')
model.add_boundary(model.metabolites.SAM, type='demand')
model.add_boundary(model.metabolites.SAH, type='demand')

Reaction identifier,DM_SAH
Name,S-Adenosyl-L-homocysteine demand
Memory address,0x07f705c38d198
Stoichiometry,SAH --> S-Adenosyl-L-homocysteine -->
GPR,
Lower bound,0
Upper bound,1000.0


In [220]:
with model:
    model.objective = model.reactions.PsiM
    max_4 = model.optimize().objective_value
    
max_4

3.139580862000786

### Changing the carbon source: 

Kan prøve at lave med LB, men så skal jeg prøve at 

Kan også bare prøve at ændre carbon source og se hvad er så sker 

In [164]:
model.medium

{'BOUNDARY_GLCe': 10.0,
 'BOUNDARY_H2Oe': 1000.0,
 'BOUNDARY_HNO3e': 1000.0,
 'BOUNDARY_O2e': 1000.0,
 'BOUNDARY_PIe': 1000.0,
 'BOUNDARY_SLFe': 1000.0,
 'BOUNDARY_Ke': 1000.0,
 'BOUNDARY_CAe': 1000.0}

In [224]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_SUCCe'] = 10
    model.medium = medium
    solution2 = model.optimize()
    print(solution2.fluxes['DRAIN_Biomass'])
    
    print(solution2.fluxes['PsiM']) # i would have like this to function but it doesnt
    

    model.objective = model.reactions.PsiM
    psi_production2 = model.optimize().objective_value
    print(psi_production2)

0.5093838996124338
0.0
1.6216216216215553


Note for me: see below that the result does not change even thouhg i put in in a seperate "with model" code:

In [225]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_SUCCe'] = 10
    model.medium = medium
    model.objective = model.reactions.PsiM
    psi_production2 = model.optimize().objective_value
    print(psi_production2)

1.6216216216215553


In [226]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_FRUe'] = 10
    model.medium = medium
    solution3 = model.optimize()
    print(solution3.fluxes['DRAIN_Biomass'])
    
    model.medium = medium
    model.objective = model.reactions.PsiM
    psi_production3 = model.optimize().objective_value
    print(psi_production3)

0.9398547241492703
3.1395808620007815


In [228]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_FUMe'] = 10
    model.medium = medium
    solution4 = model.optimize()
    print(solution4.fluxes['DRAIN_Biomass'])
  
    model.medium = medium
    model.objective = model.reactions.PsiM
    psi_production4 = model.optimize().objective_value
    print(psi_production4)

0.46992736207460967
1.5697904310004


In [229]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_GLNe'] = 10
    model.medium = medium
    solution5 = model.optimize()
    print(solution5.fluxes['DRAIN_Biomass'])
    
    model.medium = medium
    model.objective = model.reactions.PsiM
    psi_production5 = model.optimize().objective_value
    print(psi_production5)
    

0.7825699540236831
2.797087834664154


In [230]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_GLUe'] = 10
    model.medium = medium
    solution6 = model.optimize()
    print(solution6.fluxes['DRAIN_Biomass'])
    
    model.medium = medium
    model.objective = model.reactions.PsiM
    psi_production6 = model.optimize().objective_value
    print(psi_production6)


0.7825699540236909
2.797087834664169


In [231]:
medium = model.medium
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_LACe'] = 10
    model.medium = medium
    solution7 = model.optimize()
    print(solution7.fluxes['DRAIN_Biomass']) 
    
    model.medium = medium
    model.objective = model.reactions.PsiM
    psi_production7 = model.optimize().objective_value
    print(psi_production7)

0.45079094136881587
1.5058652959008532


In [232]:
medium = model.medium # don't delete this, because then the result changes
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_MALe'] = 10
    model.medium = medium
    solution8 = model.optimize()
    print(solution8.fluxes['DRAIN_Biomass'])
    
    model.medium = medium
    model.objective = model.reactions.PsiM
    psi_production8 = model.optimize().objective_value
    print(psi_production8)

0.4699273620747224
1.5697904310004152


In [233]:
medium = model.medium # don't delete this, because then the result changes
with model:
    medium['BOUNDARY_GLCe'] = 0
    medium['BOUNDARY_PYRe'] = 10
    model.medium = medium
    solution9 = model.optimize()
    print(solution9.fluxes['DRAIN_Biomass'])
    
    model.medium = medium
    model.objective = model.reactions.PsiM
    psi_production9 = model.optimize().objective_value
    print(psi_production9)

0.3916061350621861
1.308158692500327
